### Tools

Models can request to call tools that perform tasks such as fetching data from database, searching the web, or runing code. Tools are pairing of:
 * A schema, including the name of the tool, a description, and/or argument definitions (often a JSON schema) 
 * A function or coroutine to execute

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')

from langchain.chat_models import init_chat_model
model=init_chat_model("groq:qwen/qwen3-32b")

## invoke the model
response=model.invoke("Hello, how are you?")
response.content

"<think>\nHmm, the user said 'Hello, how are you?'. I should respond warmly and politely. First, I need to acknowledge their greeting. Then, I can express my state of being (as an AI, I don't have real feelings, but I can respond in a friendly way). Next, I should show interest in the user by asking how they're doing. Also, I need to let them know I'm here to help. The tone should be friendly and approachable, not too formal. Maybe add an emoji to make it more lively. Keep the response concise but not too short. Make sure to follow the rules of being helpful and friendly. Alright, that should cover it.\n</think>\n\nHello! I'm doing well, thank you for asking! 😊 How can I assist you today? I'm always here to help with any questions or tasks you might have. What's on your mind?"

In [2]:
from langchain.tools import tool

@tool
def get_current_weather(location: str) -> str:
    """Get the current weather in a given location"""
    return f"The current weather in {location} is 72 degrees and sunny."

model_with_tools = model.bind_tools([get_current_weather])

In [3]:
response = model_with_tools.invoke("What's the weather like in New York?")
response

AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user is asking about the weather in New York. I need to use the get_current_weather function. Let me check the function parameters. It requires a location, which is provided as "New York". So I\'ll call the function with that location. Make sure the JSON is correctly formatted with the name and arguments.\n', 'tool_calls': [{'id': 'tykm4a7x7', 'function': {'arguments': '{"location":"New York"}', 'name': 'get_current_weather'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 92, 'prompt_tokens': 158, 'total_tokens': 250, 'completion_time': 0.134046757, 'completion_tokens_details': {'reasoning_tokens': 66}, 'prompt_time': 0.006350988, 'prompt_tokens_details': None, 'queue_time': 0.159700272, 'total_time': 0.140397745}, 'model_name': 'qwen/qwen3-32b', 'system_fingerprint': 'fp_2bfcc54d36', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'g

### Tool Execution Loops

In [10]:
tools = {
    "get_current_weather": get_current_weather
}

# Step 1: Model generates tool calls
messages= [{"role": "user", "content": "What's the weather like in New York?"}]
ai_message = model_with_tools.invoke(messages)
messages.append(ai_message)

#Step 2: Execute tool calls and collect tool results
for tool_call in ai_message.tool_calls:
    tool_name = tool_call["name"]
    tool_args = tool_call["args"]
    tool_fn = tools[tool_name]
    tool_result=tool_fn.invoke(tool_call)
    messages.append({
        "role": "tool",
        "tool_call_id": tool_call["id"],
        "content": tool_result
    })

#Step 3: Pass results back to model to generate final response
final_response=model_with_tools.invoke(messages)
print(final_response.content)

The current weather in New York is **72 degrees** and **sunny**. Let me know if you'd like more details! ☀️
